In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import config, convert2yolov1
from src.util import video2images, get_video_folder_paths, seleccionar_imagen, merge_datasets
#from src.model import autolabel_images, predict_and_visualice
from src.dataset_processing import run_processing_dataset
import os
import json
import supervision as sv
import cv2
from src.data_augmentation import augment_dataset

## Run pipeline autolabeling

In [ ]:
import autolabeling
from tqdm import tqdm

video_paths = get_video_folder_paths(config.VIDEO_DIR_PATH)
for video_path in tqdm(video_paths):
    autolabeling.mypipeline(
        video_path=video_path,
        image_dir_path=config.IMAGE_DIR_PATH,
        frame_rate=80,
        ontology=json.load(open(os.path.join(video_paths[0], "ontology.json"))),
        output_images=config.DATASET_DIR_PATH,
        extension=".jpg",
        box_threshold =0.35,
        text_threshold = 0.25,
        num_datasets=1,
        height = None,
        width = None,
        use_yolo = True,
        not_val=True,
        model_path = "trained_models/yolov8m_640x480_cf_9cam_v34.pt",
        confidence=0.4,
        iou=0.7,
        imgsz=640,
    ) 

In [3]:
# Si solo tienes una carpeta de imágenes
from src.model import autolabel_images
autolabel_images(
            input_folder="./images/Train_linea_de_caja_640x480",
            ontology=json.load(open(os.path.join("./images/Train_linea_de_caja_640x480", "ontology.json"))),
            box_threshold=0.35,
            text_threshold=0.25,
            output_folder="./dataset",
            extension=".jpg",
            num_datasets=1,
        )

trying to load grounding dino directly


d:\Alexander\Go2Future\Autolabel_roboflow\.venv\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Labeling ./images/Train_linea_de_caja_640x480\camera65_640x480_20240311_105646-00000.jpg for dataset 1:   0%|          | 0/4 [00:00<?, ?it/s]d:\Alexander\Go2Future\Autolabel_roboflow\.venv\Lib\site-packages\transformers\modeling_utils.py:881: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
d:\Alexander\Go2Future\Autolabel_roboflow\.venv\Lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
Labeling ./images/Train_linea_de_caja_640x480\camera65_640x480_20240311_105646-00003.jpg for dataset 1: 100%|██████████| 4/4 [02:03<00:00, 30.89s/it]


Labeled dataset 1 created - ready for distillation.


# Run pipeline autotracking

In [ ]:
import autotracking
from tqdm import tqdm
from src.dataset_processing import run_processing_dataset

video_paths = get_video_folder_paths(config.VIDEO_DIR_PATH)
for video_path in tqdm(video_paths):
            autotracking.mypipeline_track(
                video_path,
                image_dir_path=config.IMAGE_DIR_PATH,
                frame_rate=40,
                output_images=config.DATASET_DIR_PATH,
                extension=".jpg",
                height = None,
                width = None,
                model_path = "trained_models/yolov8m_640x480_cf_9cam_v34.pt",
                confidence=0.4,
                iou=0.7,
                imgsz=640,

            )
# Procesar Merged_Dataset/train para eliminar errores de anotación
contenido = os.listdir(config.DATASET_DIR_PATH)
print("Contenido \n", contenido)

for data_p in contenido:
        output_path = os.path.join(config.DATASET_DIR_PATH, data_p)
        run_processing_dataset(
            os.path.join(output_path, "train", "images"),
            os.path.join(output_path, "train", "labels"),
            max_size=0.8,
            min_size=0.1,
            iou_threshold=0.7,
            remove_empty=False,
            remove_large=False,
            remove_small=False,
            remove_overlapping=False,
            remove_multiple=False,
        )

# Hacer merged de datasets

In [26]:
# Unir datasets
# crear el path a la carpeta de merged dataset dentro de dataset
output_path = os.path.join(config.DATASET_DIR_PATH, "Merged_Dataset")
    # obtener el nombre de cada dataset individual dentro de dataset
folders = os.listdir(config.DATASET_DIR_PATH)
    # Lista de paths de cada dataset individual
dataset_paths = [os.path.join(config.DATASET_DIR_PATH, folder) for folder in folders]
    # Hacer el merge
merge_datasets(dataset_paths, output_path, val=False) # val = False para autotracking

INFO:root:Merging datasets...
100%|██████████| 9/9 [00:00<00:00, 35.57it/s]
INFO:root:Merged datasets created at d:\Alexander\Go2Future\Autolabel_roboflow\dataset\Merged_Dataset


['person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person']
['person']


# Remover etiquetas incorrectas

In [9]:
# Procesar Merged_Dataset/train para eliminar errores de anotación
output_path = os.path.join(config.DATASET_DIR_PATH, "Merged_Dataset")
run_processing_dataset(
        os.path.join(output_path, "train", "images"),
        os.path.join(output_path, "train", "labels"),
        min_size=0.1,
        max_size=0.8,
        iou_threshold=0.7,
        remove_empty= True,
        remove_large= False,
        remove_small= False,
        remove_overlapping= False,
        remove_multiple= False,
    )
# Procesar Merged_Dataset/valid para eliminar errores de anotación
run_processing_dataset(
        os.path.join(output_path, "valid", "images"),
        os.path.join(output_path, "valid", "labels"),
        min_size=0.1,
        max_size=0.8,
        iou_threshold=0.7,
        remove_empty= True,
        remove_large= False,
        remove_small= False,
        remove_overlapping= False,
        remove_multiple= False,
    )



removed empty
removed empty


# Hacer data augmentation

In [ ]:
dataset_path=os.path.join(config.DATASET_DIR_PATH, "cam52")
augmented_dataset_path=os.path.join(config.DATASET_DIR_PATH, "Augmented_Dataset")
# creating aumented datased directory
if not os.path.exists(augmented_dataset_path):
    os.makedirs(augmented_dataset_path)

augment_dataset(dataset_path, augmented_dataset_path, just_rezize=False, augmented_for=5, height=480, width=640)

# Convertir a formato YOLOv1 de CVAT

In [4]:
# Convertir un solo dataset dentro de DATASET_DIR_PATH
dataset_path=os.path.join(config.DATASET_DIR_PATH, "Train_linea_de_caja_640x480_1")
convert2yolov1.convert_to_yolov1_format(dataset_path, with_val=True) 

In [ ]:
# Convertir todos los datasets dentro de un directorio a formato YOLOv1
# Directorio que contiene todos los datasets
dataset_dir_path = config.DATASET_DIR_PATH

# Iterar a través de los subdirectorios en DATASET_DIR_PATH
for dataset_name in os.listdir(dataset_dir_path):
    dataset_path = os.path.join(dataset_dir_path, dataset_name)
    
    # Verificar si el elemento en DATASET_DIR_PATH es un directorio
    if os.path.isdir(dataset_path):
        print(f"Procesando el dataset: {dataset_name}")
        convert2yolov1.convert_to_yolov1_format(dataset_path, with_val=False)

print("Proceso de conversión completado para todos los datasets.")

# Convertir anotaciones de Yolov1 CVAT a Yolov8 

In [6]:
# crear un directorio dentro de config.DATASET_DIR_PATH
yolo8_out =os.path.join(config.DATASET_DIR_PATH,"CF_9cam_with_val_640x480_corrected")
os.makedirs(yolo8_out, exist_ok=True)

convert2yolov1.convert_to_yolov8_format(os.path.join(config.DATASET_DIR_PATH,"CF_9cam_with_val_640x480_corrected_YOLOV1.zip"),yolo8_out)

# Visualizar anotaciones en formato YOLO

In [21]:
import visualizar_anotaciones

# Permitir al usuario seleccionar el directorio
directorio_seleccionado = visualizar_anotaciones.seleccionar_directorio()
if not directorio_seleccionado:
        print("No se ha seleccionado un directorio. Saliendo.")
        exit()

    # Obtener los paths de las carpetas "images" y "labels"
directorio_imagenes = os.path.join(directorio_seleccionado, "images")
directorio_labels = os.path.join(directorio_seleccionado, "labels")

# Verificar la existencia de las carpetas "images" y "labels"
if not os.path.exists(directorio_imagenes) or not os.path.exists(directorio_labels):
        print(
            "No se encontraron las carpetas 'images' y 'labels'. Asegúrate de que la estructura sea correcta."
        )
        exit()

# Ejecutar la función con los directorios proporcionados
visualizar_anotaciones.mostrar_imagenes_y_anotaciones(directorio_imagenes, directorio_labels)

Presiona 'q' para salir.


# Procesar dataset de ReID

In [2]:
# Crear Formato Market1501 a partir de CVAT for images 1.1
from src import crear_market1501
import os

xml_dir = "./dataset/Pilar_11cam_ReID"  # "D:\Alexander\Go2Future\Autolabel_roboflow\dataset\cvat_3cam_3id"
xml_file = "annotations.xml"
xml_path = os.path.join(xml_dir, xml_file)
output_dir = "default"
print(xml_path)

# obtener el nombre de la carpeta de xml_dir
output_folder = os.path.basename(xml_dir)

if output_dir == "default":
    output_dir = os.path.join(xml_dir, f"{output_folder}_market_format")
else:
    output_dir = os.path.join(output_dir, f"{output_folder}_market_format")

cvat_annotations = crear_market1501.parse_cvat_annotations(xml_path)
crear_market1501.process_cvat_annotations(
    cvat_annotations,
    output_dir,
    path_to_images=os.path.join(xml_dir, "images", "train"),
)

./dataset/Pilar_11cam_ReID\annotations.xml


In [ ]:
# Conformar las carpetas train, test y query
from src import split_train_test_query

dataset_path = "./dataset/Pilar_11cam_ReID/Pilar_11cam_ReID_market_format"  # Reemplazar con la ruta de tu dataset
output_path = "./dataset/Pilar_11cam_ReID"  # Reemplazar con la ruta deseada para el nuevo dataset

split_train_test_query.split_dataset(dataset_path, output_path)